In [ ]:
# load the data
import numpy as np
import imageio
import glob
import os

path = "./data_t_v0/2024-12-06__16-33-06/img"

# Get and print the current working directory
current_path = os.getcwd()
print(f"Current working directory: {current_path}")


# Define the path

# Get a sorted list of all PNG files in the directory
png_files = sorted(glob.glob(os.path.join(path, "*.png")))
print("png_files", png_files)

# Load the images into a list
images = [imageio.imread(file) for file in png_files]

# load

# generate video
imageio.mimsave("tmp.mp4", images, fps=10.0, codec="h264")

In [ ]:
# load the data
import json
from FlexivPy.robot.model.pinocchio import FlexivModel
import numpy as np
import pathlib
import pickle
import glob
import os
import imageio


data_bases = [ 
    "data_t_v0/2024-12-10__17-03-30" , 
    "data_t_v0/2024-12-10__17-29-36" , 
    "data_t_v0/2024-12-10__17-45-36" , 
    "data_t_v0/2024-12-11__16-15-06" , 
    "data_t_v0/2024-12-11__16-58-56" ]
              
for data_base in data_bases: 


    # Define the path

    img_path = data_base + "/img"

    # Get a sorted list of all PNG files in the directory
    png_files = sorted(glob.glob(os.path.join(img_path, "*.png")))
    print("len png_files", len(png_files))

    # Load the images into a list
    images = [imageio.imread(file) for file in png_files]

    # load 
    vid_path = pathlib.Path(data_base) / "vid"
    vid_path.mkdir(parents=True, exist_ok=True)
    fout = str(vid_path / "vid.mp4")
    # generate video 
    imageio.mimsave(fout , images  , fps=10., codec = "h264")
    print("saving video" ,fout )

    data_file = data_base +  "/data/data.pkl"
    with open(data_file, "rb") as f:
        Din = pickle.load(f)
    model = FlexivModel()

    # compute the end effector position. 
    def get_endeff_pos(q):
        dq = np.zeros(7)
        D = model.getInfo(np.array(q), dq)
        T0 =  D["poses"]["link7"]
        return T0[:2,3]

    ps = [ get_endeff_pos(q) for q in Din["q"]  ]
    dt = .1
    vs = [ (ps[i+1] - ps[i]) / dt for i in range(len(ps) - 1) ]

    Dout = Din
    Dout["ps"] =  [ p.tolist() for p in  ps  ] 
    Dout["vs"] =  [  v.tolist() for v in  vs   ]


    data_file = data_base + "/data/data_extended.pkl"
    with open(data_file, "wb") as f:
        pickle.dump(Dout, f)







In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

# Example data (replace with your own)
# Suppose we have 50 frames
num_frames = len(images) - 1
num_frames = 2000

# images = [np.random.rand(100, 100) for _ in range(num_frames)]  # List of 2D arrays
# positions = np.linspace(0, 100, num_frames)  # Example scalar positions
# velocities = np.sin(np.linspace(0, 2*np.pi, num_frames)) * 10  # Example scalar velocities
positions_x = [p[0] for p in ps ]
positions_y = [p[1] for p in ps ]

min_pos = min(positions_x + positions_y)
max_pos = max(positions_x + positions_y)

velocities_x = [v[0] for v in vs ]
velocities_y = [v[1] for v in vs ]

min_vel = min(velocities_y + velocities_y)
max_vel = max(velocities_x + velocities_y)


# Parameters
history_length = 10  # number of last measurements to show

# Create figure and subplots
fig = plt.figure(figsize=(10, 5))
# GridSpec to organize:
# Left half: image
# Right half: two subplots stacked vertically for position and velocity
gs = fig.add_gridspec(2, 2, width_ratios=[1.5, 1], height_ratios=[1, 1])

ax_img = fig.add_subplot(gs[:, 0])           # Large subplot on the left for the image
ax_pos = fig.add_subplot(gs[0, 1])           # Top-right subplot for position
ax_vel = fig.add_subplot(gs[1, 1])           # Bottom-right subplot for velocity

# Initialize plots
img_plot = ax_img.imshow(images[0], cmap='gray', vmin=0, vmax=1)
ax_img.set_title("Frame 0")
ax_img.axis('off')

pos_line_x, = ax_pos.plot([], [], marker='o', linestyle='-', color='blue')
pos_line_y, = ax_pos.plot([], [], marker='o', linestyle='-', color='red')

ax_pos.set_title("Last 10 Positions")
ax_pos.set_xlim(0, history_length * 2)
ax_pos.set_ylim(min_pos, max_pos)
ax_pos.grid(True)

vel_line_x, = ax_vel.plot([], [], marker='o', linestyle='-', color='blue')
vel_line_y, = ax_vel.plot([], [], marker='o', linestyle='-', color='red')

ax_vel.set_title("Last 10 Velocities")
ax_vel.set_xlim(0, history_length * 2)
ax_vel.set_ylim( min_vel, max_vel)
ax_vel.grid(True)

def init():
    # Initialization function for FuncAnimation
    img_plot.set_data(images[0])
    pos_line_x.set_data([], [])
    pos_line_y.set_data([], [])
    vel_line_x.set_data([], [])
    vel_line_y.set_data([], [])
    return img_plot, pos_line_x, pos_line_y, vel_line_x , vel_line_y

def update(frame):
    # Update image
    img_plot.set_data(images[frame])
    ax_img.set_title(f"Frame {frame}")

    # Update position data (last 10 measurements)
    start_idx = max(0, frame - history_length + 1)
    pos_line_x.set_data(range(0 , frame+1 -start_idx ), positions_x[start_idx:frame+1])
    pos_line_y.set_data(range(0 , frame+1 -start_idx ), positions_y[start_idx:frame+1])

    
    
    # Update velocity data (last 10 measurements)
    vel_line_x.set_data(range(0 , frame+1 -start_idx ), velocities_x[start_idx:frame+1])
    vel_line_y.set_data(range(0 , frame+1 -start_idx ), velocities_y[start_idx:frame+1])

    
    return img_plot, pos_line_x ,  pos_line_y ,  vel_line_x ,  vel_line_y 

ani = animation.FuncAnimation(
    fig, update, frames=num_frames, init_func=init, blit=False, interval= 1 / dt
)

# Uncomment below to save to a video file (requires ffmpeg)
ani.save('output_video.mp4', fps=10, extra_args=['-vcodec', 'libx264'])

plt.tight_layout()
plt.show()


In [ ]:
print([a-b for a,b in zip(positions_x,positions_y) ] )